In [34]:
import tensorflow as tf
import time
import numpy as np
import os
import copy
import pickle
import argparse
import utilityarm as utility
import pandas as pd
from sklearn.metrics import *
import tensorflow.keras.backend as K

In [35]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior() 


class FairAdvBPR:

    def __init__(self, sess, dict_args, train_df, test_df, user_type, type_error_weight, key_type, user_type_list, item_type_count, adv_const):
       
        self.dataname = dict_args['dataname']

        self.key_type = key_type
        self.user_type_list = user_type_list
        self.item_type_count = item_type_count
        self.layers = dict_args['layers']
        self.sess = sess
        
        self.num_cols = len(train_df['item_id'].unique())
        self.num_rows = len(train_df['user_id'].unique())

        self.hidden_neuron = dict_args['hidden_neuron']
        self.neg = dict_args['neg']
        self.batch_size = dict_args['batch_size']

        self.train_df = train_df
        self.vali_df = test_df
        self.num_train = len(self.train_df)
        self.num_vali = len(self.vali_df)
        self.adv_const = adv_const
        
        self.train_epoch = dict_args['train_epoch']
        self.train_epoch_a = dict_args['train_epoch_a']
       # self.train_epoch_a_post = dict_args['train_epoch_a_post']
        self.train_epoch_all = dict_args['train_epoch_all']

        self.lr_r = dict_args['lr_r'] # learning rate
        self.lr_a = dict_args['lr_a'] # learning rate
        self.alpha = dict_args['alpha'] # learning rate
        self.optimizer_method = dict_args['optimizer_method']
        self.display_step = dict_args['display_step']
        
        self.type_error_weight = type_error_weight
        self.num_type = dict_args['num_type']
        
        self.user_type = user_type
        self.type_count_list = []
        for k in range(self.num_type):
            self.type_count_list.append(np.sum(user_type[:,k]))

        
        self.reg = dict_args['reg'] # regularization term trade-off
        self.reg_s = dict_args['reg_s']

        print('**********fairAdvBPR**********')
        #print(self.args)
        self._prepare_model()

    def loadmodel(self, saver, checkpoint_dir):
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            return True
        else:
            return False
        
    def run(self):
        init = tf.global_variables_initializer()
        self.sess.run(init)

        saver = tf.train.Saver([self.P, self.Q])
        self.loadmodel(saver, "./"+self.dataname+"/BPR_check_points")

        for epoch_itr in range(1, self.train_epoch + 1 + self.train_epoch_a+ self.train_epoch_all):
            self.train_model(epoch_itr)
            if epoch_itr % self.display_step == 0:
                self.test_model(epoch_itr)
        return self.make_records()

    def _prepare_model(self):
        with tf.name_scope("input_data"):
            self.user_input = tf.placeholder(tf.int32, shape=[None, 1], name="user_input")
            self.item_input_pos = tf.placeholder(tf.int32, shape=[None, 1], name="item_input_pos")
            self.item_input_neg = tf.placeholder(tf.int32, shape=[None, 1], name="item_input_neg")

            self.input_user_type = tf.placeholder(dtype=tf.float32, shape=[None, self.num_type]
                                                   , name="input_user_type")
            self.input_user_error_weight = tf.placeholder(dtype=tf.float32, shape=[None, 1]
                                                          , name="input_user_error_weight")

        with tf.variable_scope("BPR", reuse=tf.AUTO_REUSE):
            self.P = tf.get_variable(name="P",
                                     initializer=tf.truncated_normal(shape=[self.num_rows, self.hidden_neuron], mean=0,
                                                                     stddev=0.03), dtype=tf.float32)
            self.Q = tf.get_variable(name="Q",
                                     initializer=tf.truncated_normal(shape=[self.num_cols+3, self.hidden_neuron], mean=0,
                                                                     stddev=0.03), dtype=tf.float32)
        para_r = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="BPR")

        with tf.variable_scope("Adversarial", reuse=tf.AUTO_REUSE):
            num_layer = len(self.layers)
            adv_W = []
            adv_b = []
            for l in range(num_layer):
                if l == 0:
                    in_shape = 65
                else:
                    in_shape = self.layers[l - 1]
                adv_W.append(tf.get_variable(name="adv_W" + str(l),
                                             initializer=tf.truncated_normal(shape=[in_shape, self.layers[l]],
                                                                             mean=0, stddev=0.03), dtype=tf.float32))
                adv_b.append(tf.get_variable(name="adv_b" + str(l),
                                             initializer=tf.truncated_normal(shape=[1, self.layers[l]],
                                                                             mean=0, stddev=0.03), dtype=tf.float32))
            adv_W_out = tf.get_variable(name="adv_W_out",
                                        initializer=tf.truncated_normal(shape=[self.layers[-1], self.num_type],
                                                                        mean=0, stddev=0.03), dtype=tf.float32)

            adv_b_out = tf.get_variable(name="adv_b_out",
                                        initializer=tf.truncated_normal(shape=[1, self.num_type],
                                                                        mean=0, stddev=0.03), dtype=tf.float32)
        para_a = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="Adversarial")

        p = tf.reduce_sum(tf.nn.embedding_lookup(self.P, self.user_input), 1)
        q_neg = tf.reduce_sum(tf.nn.embedding_lookup(self.Q, self.item_input_neg), 1)
        q_pos = tf.reduce_sum(tf.nn.embedding_lookup(self.Q, self.item_input_pos), 1)

        predict_pos = tf.reduce_sum(p * q_pos, 1)
        predict_neg = tf.reduce_sum(p * q_neg, 1)

        r_cost1 = tf.reduce_sum(tf.nn.softplus(-(predict_pos - predict_neg)))
        r_cost2 = self.reg * 0.5 * (self.l2_norm(self.P) + self.l2_norm(self.Q))  # regularization term
#         pred = tf.matmul(self.P, tf.transpose(self.Q))
#         self.s_mean = tf.reduce_mean(pred, axis=1)
#         self.s_std = tf.keras.backend.std(pred, axis=1)
#         self.s_cost = tf.reduce_sum(tf.square(self.s_mean) + tf.square(self.s_std) - 2 * tf.log(self.s_std) - 1)#additional regularization
        self.s_mean = 0
        self.s_std = 0
        self.s_cost = 0
        
#        print('input_user_type ',self.input_user_type[:,1].shape)
#         usertype = self.input_user_type[]
#         print('shape',usertype.shape)
#         const_user_type1 = tf.reduce_sum(tf.nn.softplus(-(predict_pos - predict_neg) * self.input_user_type[:,0]))
#         const_user_type2 = tf.reduce_sum(tf.nn.softplus(-(predict_pos - predict_neg) * self.input_user_type[:,1]))
        
#        const = K.sqrt(K.square(const_user_type1 - const_user_type2))
        
#        self.r_cost = r_cost1 + r_cost2 + 1 * const + self.reg_s * 0.5 * self.s_cost
        self.r_cost = r_cost1 + r_cost2 
        
        print('shape q pos',q_pos.shape)
        print('shape p',p.shape)
        print('shape predict pos ', predict_pos.shape)
        
        adv_last = tf.reshape(predict_pos, [tf.shape(self.input_user_type)[0], 1])
        print('shape adv_last ', adv_last.shape)
        adv_last = tf.concat([adv_last, p], 1)
        
        for l in range(num_layer):
            adv = tf.nn.relu(tf.matmul(adv_last, adv_W[l]) + adv_b[l])
            adv_last = adv
        self.adv_output = tf.nn.softmax(tf.matmul(adv_last, adv_W_out) + adv_b_out)
 #       self.a_cost = tf.reduce_sum(tf.square(self.adv_output - self.input_user_type) * self.input_user_error_weight)
        self.a_cost = tf.reduce_sum(- self.input_user_type * tf.math.log(self.adv_output))

    
                  

 #       self.all_cost = self.r_cost - self.alpha * self.a_cost  # the loss function
        self.all_cost = self.r_cost - self.adv_const * self.a_cost  # the loss function

        with tf.variable_scope("Optimizer", reuse=tf.AUTO_REUSE):
            self.r_optimizer = tf.train.AdamOptimizer(learning_rate=self.lr_r).minimize(self.r_cost, var_list=para_r)
            self.a_optimizer = tf.train.AdamOptimizer(learning_rate=self.lr_a).minimize(self.a_cost, var_list=para_a)
            self.all_optimizer = tf.train.AdamOptimizer(learning_rate=self.lr_r).minimize(self.all_cost, var_list=para_r)


    def train_model(self, itr):
        NS_start_time = time.time() * 1000.0
        epoch_r_cost = 0.0
        epoch_s_cost = 0.0
        epoch_s_mean = 0.0
        epoch_s_std = 0.0
        epoch_a_cost = 0.0
        accuracy_adv_pre_temp = 0.0
        accuracy_adv_post_temp = 0.0
        accuracy_adv_pre =[]
        accuracy_adv_post =[]
        self.accuracy_adv_pre = [0]
        epoch_adv_acc_pre = 0.0
        epoch_adv_acc_post = 0.0
        num_sample, user_list, item_pos_list, item_neg_list = utility.negative_sample(self.train_df, self.num_rows,
                                                                                      self.num_cols, self.neg)
        NS_end_time = time.time() * 1000.0

        start_time = time.time() * 1000.0
        num_batch = int(num_sample / float(self.batch_size)) + 1
        random_idx = np.random.permutation(num_sample)
        for i in range(num_batch):
            # get the indices of the current batch
            if i == num_batch - 1:
                batch_idx = random_idx[i * self.batch_size:]
            elif i < num_batch - 1:
                batch_idx = random_idx[(i * self.batch_size):((i + 1) * self.batch_size)]
            
            accuracy_adv_pre_temp = 0.0
            accuracy_adv_post_temp = 0.0
            # training of adversary
            if itr > self.train_epoch and itr < (self.train_epoch + self.train_epoch_a + 1):
                #random_idx_a = np.random.permutation(num_sample)
                #print("boucle adversarial debut-- num batch ",i)
                #for j in range(num_batch):
#                 if j == num_batch - 1:
#                 batch_idx_a = random_idx_a[j * self.batch_size:]
#                 elif j < num_batch - 1:
#                 batch_idx_a = random_idx_a[(j * self.batch_size):((j + 1) * self.batch_size)]
                user_idx_list = ((user_list[batch_idx, :]).reshape((len(batch_idx)))).tolist()
                _, tmp_a_cost, adv_output = self.sess.run(  # do the optimization by the minibatch
                        [self.a_optimizer, self.a_cost, self.adv_output],
                feed_dict={self.user_input: user_list[batch_idx, :],
                                   self.item_input_pos: item_pos_list[batch_idx, :],
                                   self.item_input_neg: item_neg_list[batch_idx, :],
                                   self.input_user_type: self.user_type[user_idx_list,:],
                                   self.input_user_error_weight: self.type_error_weight[user_idx_list,:]})
                epoch_a_cost += tmp_a_cost
                    
                g = tf.Graph()
                with g.as_default():
                    logits = adv_output
                      
                    labels = self.user_type[user_idx_list,:]
                      
                    acc, acc_op = tf.compat.v1.metrics.accuracy(labels, logits)
                    global_init = tf.compat.v1.global_variables_initializer()
                    local_init = tf.compat.v1.local_variables_initializer()
                sess = tf.compat.v1.Session(graph=g)
                sess.run([global_init, local_init])
                acc, acc_op = sess.run([acc, acc_op])
                    #print('post adversarial -- adversaire ready ',acc_op)
                accuracy_adv_pre_temp += acc_op
                sess.close()
                accuracy_adv_pre.append(accuracy_adv_pre_temp)
                print('post adversarial -- adversaire ready after one adv epoch training ',accuracy_adv_pre_temp)
                
                if itr == self.train_epoch + self.train_epoch_a:
                    self.accuracy_adv_pre.append(accuracy_adv_pre_temp)
                
            if itr > (self.train_epoch + self.train_epoch_a ):  
                
                user_idx_list = ((user_list[batch_idx, :]).reshape((len(batch_idx)))).tolist()
                _, tmp_r_cost = self.sess.run(  # do the optimization by the minibatch
                    [self.all_optimizer, self.all_cost],
                    feed_dict={self.user_input: user_list[batch_idx, :],
                               self.item_input_pos: item_pos_list[batch_idx, :],
                               self.item_input_neg: item_neg_list[batch_idx, :],
                               self.input_user_type: self.user_type[user_idx_list, :],
                               self.input_user_error_weight: self.type_error_weight[user_idx_list, :]})
                epoch_r_cost += tmp_r_cost
                #mesure perf adv post training of main model
#                 for j in range(num_batch):
#                     if j == num_batch - 1:
#                         batch_idx_a = random_idx_a[j * self.batch_size:]
#                     elif j < num_batch - 1:
#                         batch_idx_a = random_idx_a[(j * self.batch_size):((j + 1) * self.batch_size)]
            
             
                user_idx_list = ((user_list[batch_idx, :]).reshape((len(batch_idx)))).tolist()
#                 _,tmp_a_cost, adv_output_post = self.sess.run(  # do the optimization by the minibatch
#                          [self.a_optimizer, self.a_cost, self.adv_output], feed_dict={self.user_input: user_list[batch_idx, :],
#                                     self.item_input_pos: item_pos_list[batch_idx, :],
#                                     self.item_input_neg: item_neg_list[batch_idx, :],
#                                     self.input_user_type: self.user_type[user_idx_list,:],
#                                     self.input_user_error_weight: self.type_error_weight[user_idx_list,:]})
#                 epoch_a_cost += tmp_a_cost
                
                adv_output_post = self.sess.run(  # do the optimization by the minibatch
                        [ self.adv_output],
                feed_dict={self.user_input: user_list[batch_idx, :],
                                   self.item_input_pos: item_pos_list[batch_idx, :],
                                   self.item_input_neg: item_neg_list[batch_idx, :],
                                   self.input_user_type: self.user_type[user_idx_list,:],
                                   self.input_user_error_weight: self.type_error_weight[user_idx_list,:]})
                    
                g = tf.Graph()
                with g.as_default():
                    labels2 = self.user_type[user_idx_list,:] 
                    logits2 = adv_output_post                    
                    acc, acc_op = tf.compat.v1.metrics.accuracy(labels2, logits2[0])
                    global_init = tf.compat.v1.global_variables_initializer()
                    local_init = tf.compat.v1.local_variables_initializer()
                sess = tf.compat.v1.Session(graph=g)
                sess.run([global_init, local_init])
                acc, acc_op = sess.run([acc, acc_op])
                    #print('post main model training -- adversaire dejoue', acc_op)
                accuracy_adv_post_temp += acc_op
                sess.close()
                accuracy_adv_post.append(accuracy_adv_post_temp)
                print('post adversarial -- adversaire dejoue after one re-training of main model ',accuracy_adv_post_temp)
                
                
                print("boucle adversarial fin")
            else:
                user_idx_list = ((user_list[batch_idx, :]).reshape((len(batch_idx)))).tolist()
                _, tmp_r_cost = self.sess.run(  # do the optimization by the minibatch
                    [self.r_optimizer, self.r_cost],
                    feed_dict={self.user_input: user_list[batch_idx, :],
                               self.item_input_pos: item_pos_list[batch_idx, :],
                               self.item_input_neg: item_neg_list[batch_idx, :],
                               self.input_user_type: self.user_type[user_idx_list, :],
                               self.input_user_error_weight: self.type_error_weight[user_idx_list, :]})
                epoch_r_cost += tmp_r_cost
    
        epoch_a_cost /= num_batch
        epoch_adv_acc_pre = sum(accuracy_adv_pre)/num_batch
        epoch_adv_acc_post = sum(accuracy_adv_post)/num_batch
        epoch_adv_acc_pre_lastAdvIter = sum(self.accuracy_adv_pre)/num_batch
        
        
        filename = './const_occu_UembfairAdvBPRresultsAdvConst'+str(self.adv_const)+ '/epoch'+ str(itr) +'_pre_' + self.dataname + '_accAdvpre.npy'
        os.makedirs(os.path.dirname(filename), exist_ok=True)           
        with open(filename, "wb") as f:
                np.save(f, epoch_adv_acc_pre) 
        filename1 = './const_occu_UembfairAdvBPRresultsAdvConst'+str(self.adv_const)+ '/epoch'+ str(itr) +'_post_' + self.dataname + '_accAdvpost.npy'
        os.makedirs(os.path.dirname(filename1), exist_ok=True)           
        with open(filename1, "wb") as f:
                np.save(f, epoch_adv_acc_post) 
                
        if itr % self.display_step == 0:
            print ("Training //", "Epoch %d //" % itr, " Total r_cost = %.5f" % epoch_r_cost,
                   " Total a_cost = %.5f" % epoch_a_cost,
                   "total pre adversarial accuracy = %.5f" % epoch_adv_acc_pre,
                   "total pre adversarial accuracy = %.5f" % epoch_adv_acc_pre_lastAdvIter,
                   "total post adversarial accuracy = %.5f" % epoch_adv_acc_post,
                   "Training time : %d ms" % (time.time() * 1000.0 - start_time),
                   "negative Sampling time : %d ms" % (NS_end_time - NS_start_time),
                   "negative samples : %d" % (num_sample))
       
    def test_model(self, itr):  # calculate the cost and rmse of testing set in each epoch
        if itr % self.display_step == 0:
            start_time = time.time() * 1000.0
            P, Q = self.sess.run([self.P, self.Q])
            Rec = np.matmul(P, Q.T)

            [precision, recall, f_score, NDCG] = utility.test_model_all(Rec, self.vali_df, self.train_df)
#             utility.ranking_analysis(Rec, self.vali_df, self.train_df, self.key_genre, self.item_genre_list,
#                                      self.user_genre_count)
            utility.test_model_per_user_type_multiclass(Rec, self.vali_df, self.train_df, self.user_type_list, self.key_type)
            auc = utility.auc_per_user_multiclass(Rec, self.vali_df, self.train_df)
            print("AUC global is: ", auc)
            
            filename = './const_occu_UembfairAdvBPRresultsAdvConst'+str(self.adv_const)+ '/epoch'+ str(itr) +'_Rec_' + self.dataname + '_constfairAdvBPR.npy'
            os.makedirs(os.path.dirname(filename), exist_ok=True)           
            with open(filename, "wb") as f:
                np.save(f, Rec)
                

            

    def make_records(self):  # record all the results' details into files
        P, Q = self.sess.run([self.P, self.Q])
        Rec = np.matmul(P, Q.T)

        [precision, recall, f_score, NDCG] = utility.test_model_all(Rec, self.vali_df, self.train_df)
        return precision, recall, f_score, NDCG, Rec

#     def test_model(self, itr):  # calculate the cost and rmse of testing set in each epoch
#         if itr % self.display_step == 0:
#             start_time = time.time() * 1000.0
#             P, Q = self.sess.run([self.P, self.Q])
#             Rec = np.matmul(P, Q.T)

#             [precision, recall, f_score, NDCG] = utility.test_model_all(Rec, self.vali_df, self.train_df)
# #             utility.ranking_analysis(Rec, self.vali_df, self.train_df, self.key_type, self.user_type_list,
# #                                      self.item_type_count)
#             utility.test_model_per_user_type(Rec, self.vali_df, self.train_df, self.user_type_list, self.key_type)
#             auc = utility.auc_per_user(Rec, self.vali_df, self.train_df)
#             print("AUC global is: ", auc)
#             print (
#                 "Testing //", "Epoch %d //" % itr,
#                 "Testing time : %d ms" % (time.time() * 1000.0 - start_time))
#             print("=" * 200)


    @staticmethod
    def l2_norm(tensor):
        return tf.reduce_sum(tf.square(tensor))


In [36]:

#optimizer_method = ['Adam', 'Adadelta', 'Adagrad', 'RMSProp', 'GradientDescent','Momentum'], default='Adam')


train_epoch = 10
train_epoch_a = 40 #default 20
train_epoch_all = 40
#train_epoch_a_post = 5
display_step = 1
lr_r = 0.0001
lr_a = 0.00005
reg = 0.1
reg_s = 30
alpha = 1000
optimizer_method = 'Adam'
hidden_neuron = 20
adv_const = [1000]
neg = 5
batch_size = 1024
#layers = [50, 50, 50, 50]
layers = [256]
dataname = 'ml1m-6'

In [37]:
dict_args =  {"train_epoch": train_epoch,
              "train_epoch_a": train_epoch_a,
              "train_epoch_all": train_epoch_all,
             # "train_epoch_a_post": train_epoch_a_post,
            "display_step":display_step,
            "lr_r":lr_r,
            "lr_a":lr_a,
            "reg":reg,
            "reg_s":reg_s,
            "alpha":alpha,
            "optimizer_method":optimizer_method,
            "hidden_neuron":hidden_neuron,
            "adv_const":adv_const,
            "neg":neg,
            "batch_size":batch_size,
            "layers":layers,
            "dataname":dataname}
dict_args

{'train_epoch': 10,
 'train_epoch_a': 40,
 'train_epoch_all': 40,
 'display_step': 1,
 'lr_r': 0.0001,
 'lr_a': 5e-05,
 'reg': 0.1,
 'reg_s': 30,
 'alpha': 1000,
 'optimizer_method': 'Adam',
 'hidden_neuron': 20,
 'adv_const': [1000],
 'neg': 5,
 'batch_size': 1024,
 'layers': [256],
 'dataname': 'ml1m-6'}

In [38]:
with open('./training_df.pkl', 'rb') as f:
    train_df = pickle.load(f,encoding='latin1')

# with open('./' + dataname + '/valiing_df.pkl', 'rb') as f:
#     vali_df = pickle.load(f,encoding='latin1')  # for validation
    
with open('./testing_df.pkl', 'rb') as f:
    test_df = pickle.load(f,encoding='latin1')  # for validation
# vali_df = pickle.load(open('./' + dataname + '/testing_df.pkl'))  # for testing

with open('./key_type.pkl', 'rb') as f:
    key_type = pickle.load(f,encoding='latin1')
    
with open('./user_idd_type_list.pkl', 'rb') as f:
    user_idd_type_list = pickle.load(f,encoding='latin1')
    
with open('./type_user_vector.pkl', 'rb') as f:
    type_user_vector = pickle.load(f,encoding='latin1')

with open('./type_count.pkl', 'rb') as f:
    type_count = pickle.load(f,encoding='latin1')
    
with open('./item_type_count.pkl', 'rb') as f:
    item_type_count = pickle.load(f,encoding='latin1')

In [39]:
train_df.head(20)

,user_id,item_id,rating
0,0,0,3
1,1,1,4
2,2,2,2
3,3,3,3
4,4,4,3
5,5,5,4
6,6,6,3
7,7,7,4
8,8,8,4
9,9,9,4


In [40]:
train_df.shape

(12367, 3)

In [41]:
test_df.head(20)

,user_id,item_id,rating
0,2,12,3
1,14,16,4
2,21,27,4
3,1,28,3
4,21,43,2
5,24,22,2
6,11,49,5
7,34,50,3
8,16,58,4
9,21,63,3


In [42]:
test_df.shape

(3015, 3)

In [43]:
print(len(user_idd_type_list))

158


In [44]:
user_idd_type_list

[['executive'],
 ['executive'],
 ['engineer'],
 ['executive'],
 ['engineer'],
 ['engineer'],
 ['artist'],
 ['engineer'],
 ['executive'],
 ['artist'],
 ['engineer'],
 ['artist'],
 ['engineer'],
 ['artist'],
 ['engineer'],
 ['executive'],
 ['engineer'],
 ['executive'],
 ['engineer'],
 ['executive'],
 ['executive'],
 ['engineer'],
 ['engineer'],
 ['scientist'],
 ['executive'],
 ['engineer'],
 ['scientist'],
 ['executive'],
 ['engineer'],
 ['scientist'],
 ['engineer'],
 ['executive'],
 ['executive'],
 ['engineer'],
 ['artist'],
 ['artist'],
 ['engineer'],
 ['engineer'],
 ['engineer'],
 ['artist'],
 ['engineer'],
 ['artist'],
 ['scientist'],
 ['artist'],
 ['engineer'],
 ['executive'],
 ['artist'],
 ['scientist'],
 ['artist'],
 ['scientist'],
 ['scientist'],
 ['executive'],
 ['engineer'],
 ['engineer'],
 ['scientist'],
 ['artist'],
 ['engineer'],
 ['scientist'],
 ['executive'],
 ['scientist'],
 ['scientist'],
 ['executive'],
 ['scientist'],
 ['engineer'],
 ['engineer'],
 ['executive'],
 ['en

In [45]:
type_user_vector

{'engineer': array([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
         1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
         0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
         0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1.,
         1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
         1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
         0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.]]),
 'scientist': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
         0., 1., 1., 0., 0., 0.

In [46]:
len(item_type_count)

943

In [47]:
item_type_count

[{'executive': 24, 'engineer': 50, 'artist': 19, 'scientist': 22},
 {'executive': 25, 'engineer': 52, 'artist': 22, 'scientist': 27},
 {'executive': 28, 'engineer': 42, 'artist': 27, 'scientist': 25},
 {'executive': 30, 'engineer': 53, 'artist': 28, 'scientist': 29},
 {'executive': 32, 'engineer': 60, 'artist': 28, 'scientist': 31},
 {'executive': 27, 'engineer': 48, 'artist': 25, 'scientist': 24},
 {'executive': 22, 'engineer': 51, 'artist': 20, 'scientist': 27},
 {'executive': 25, 'engineer': 53, 'artist': 25, 'scientist': 27},
 {'executive': 27, 'engineer': 60, 'artist': 26, 'scientist': 30},
 {'executive': 16, 'engineer': 60, 'artist': 25, 'scientist': 28},
 {'executive': 22, 'engineer': 32, 'artist': 15, 'scientist': 20},
 {'executive': 28, 'engineer': 50, 'artist': 24, 'scientist': 27},
 {'executive': 30, 'engineer': 63, 'artist': 25, 'scientist': 29},
 {'executive': 31, 'engineer': 64, 'artist': 23, 'scientist': 31},
 {'executive': 29, 'engineer': 61, 'artist': 26, 'scientist': 

In [48]:
print(type_count)

{'executive': 32, 'engineer': 67, 'artist': 28, 'scientist': 31}


In [49]:
num_item = len(train_df['item_id'].unique())
num_user = len(train_df['user_id'].unique())
num_type = len(key_type)
print('items number : ',num_item)
print('users number : ',num_user)
print('user types : ',key_type)


items number :  942
users number :  158
user types :  ['engineer', 'scientist', 'executive', 'artist']


In [50]:
dict_args["num_type"] = len(key_type)

In [51]:
user_type_list = [] #preprocessing to be sure that user types are really the right ones armielle 
for u in range(num_user):
    gl = user_idd_type_list[u]
    tmp = []
    for g in gl:
        if g in key_type:
            tmp.append(g)
    user_type_list.append(tmp)

print(len(user_type_list))

158


In [52]:
# genreate user_type matrix
user_type = np.zeros((num_user, num_type))
for u in range(num_user):
    gl = user_type_list[u]
    for k in range(num_type):
        if key_type[k] in gl:
            user_type[u, k] = 1.0

In [53]:
len(user_type_list)

158

In [54]:
print('*' * 50)
print('number of positive feedback: ' + str(len(train_df)))
print('estimated number of training samples: ' + str(neg * len(train_df)))
print('*' * 50)

**************************************************
number of positive feedback: 12367
estimated number of training samples: 61835
**************************************************


In [55]:
type_count_mean_reciprocal = []
for k in key_type:
    type_count_mean_reciprocal.append(1.0 / type_count[k])
type_count_mean_reciprocal = (np.array(type_count_mean_reciprocal)).reshape((num_type, 1))
type_error_weight = np.dot(user_type, type_count_mean_reciprocal)


In [56]:
# generate user_type matrix
type_user_indicator = np.zeros((num_type, num_user))

for k in range(num_type):
    type_user_indicator[k,:] = type_user_vector[key_type[k]]


In [57]:
precision = np.zeros(4)
recall = np.zeros(4)
f1 = np.zeros(4)
ndcg = np.zeros(4)
RSP = np.zeros(4)
REO = np.zeros(4)

precision 

array([0., 0., 0., 0.])

In [58]:
len(user_type)

158

In [59]:
user_type

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],


In [ ]:
for i in adv_const:
    with tf.compat.v1.Session() as sess:
        fairadvbpr = FairAdvBPR(sess, dict_args, train_df, test_df, user_type, type_error_weight, key_type, user_type_list, item_type_count, i)
        [prec_one, rec_one, f_one, ndcg_one, Rec] = fairadvbpr.run()
        #[RSP_one, REO_one] = utility.ranking_analysis(Rec, vali_df, train_df, key_genre, item_genre_list, user_genre_count)
#         precision += prec_one
#         recall += rec_one
#         f1 += f_one
#         ndcg += ndcg_one
#         RSP += RSP_one
#         REO += REO_one

**********fairAdvBPR**********
shape q pos (?, 64)
shape p (?, 64)
shape predict pos  (?,)
shape adv_last  (?, 1)
INFO:tensorflow:Restoring parameters from ./ml1m-6/BPR_check_points\check_point.ckpt-41
Training // Epoch 1 //  Total r_cost = 25281.12738  Total a_cost = 0.00000 total pre adversarial accuracy = 0.00000 total pre adversarial accuracy = 0.00000 total post adversarial accuracy = 0.00000 Training time : 112 ms negative Sampling time : 911 ms negative samples : 61835
precision_1	[0.4430380],	||	 precision_5	[0.3531646],	||	 precision_10	[0.2981013],	||	 precision_15	[0.2654008]
recall_1   	[0.0326306],	||	 recall_5   	[0.1279361],	||	 recall_10   	[0.2031681],	||	 recall_15   	[0.2622732]
f_measure_1	[0.0607843],	||	 f_measure_5	[0.1878297],	||	 f_measure_10	[0.2416452],	||	 f_measure_15	[0.2638277]
ndcg_1     	[0.4430380],	||	 ndcg_5     	[0.3755212],	||	 ndcg_10     	[0.3548082],	||	 ndcg_15     	[0.3549585]
Metrics for user type	 engineer
precision_1	[0.4776119],	||	 precis

Training // Epoch 5 //  Total r_cost = 24907.79674  Total a_cost = 0.00000 total pre adversarial accuracy = 0.00000 total pre adversarial accuracy = 0.00000 total post adversarial accuracy = 0.00000 Training time : 64 ms negative Sampling time : 843 ms negative samples : 61835
precision_1	[0.4556962],	||	 precision_5	[0.3569620],	||	 precision_10	[0.2955696],	||	 precision_15	[0.2679325]
recall_1   	[0.0357124],	||	 recall_5   	[0.1308681],	||	 recall_10   	[0.2011172],	||	 recall_15   	[0.2640447]
f_measure_1	[0.0662342],	||	 f_measure_5	[0.1915214],	||	 f_measure_10	[0.2393627],	||	 f_measure_15	[0.2659744]
ndcg_1     	[0.4556962],	||	 ndcg_5     	[0.3804639],	||	 ndcg_10     	[0.3555327],	||	 ndcg_15     	[0.3594502]
Metrics for user type	 engineer
precision_1	[0.4626866],	||	 precision_5	[0.3970149],	||	 precision_10	[0.3417910],	||	 precision_15	[0.3184080]
recall_1	[0.0259996],	||	 recall_5	[0.1075793],	||	 recall_10	[0.1761304],	||	 recall_15	[0.2430906]
ndcg_1	[0.4626866],	||	 

Training // Epoch 9 //  Total r_cost = 24497.27455  Total a_cost = 0.00000 total pre adversarial accuracy = 0.00000 total pre adversarial accuracy = 0.00000 total post adversarial accuracy = 0.00000 Training time : 72 ms negative Sampling time : 1040 ms negative samples : 61835
precision_1	[0.4620253],	||	 precision_5	[0.3670886],	||	 precision_10	[0.2962025],	||	 precision_15	[0.2691983]
recall_1   	[0.0349916],	||	 recall_5   	[0.1349017],	||	 recall_10   	[0.2012317],	||	 recall_15   	[0.2660335]
f_measure_1	[0.0650562],	||	 f_measure_5	[0.1972982],	||	 f_measure_10	[0.2396511],	||	 f_measure_15	[0.2676066]
ndcg_1     	[0.4620253],	||	 ndcg_5     	[0.3888378],	||	 ndcg_10     	[0.3574445],	||	 ndcg_15     	[0.3617059]
Metrics for user type	 engineer
precision_1	[0.4626866],	||	 precision_5	[0.4029851],	||	 precision_10	[0.3402985],	||	 precision_15	[0.3194030]
recall_1	[0.0245485],	||	 recall_5	[0.1098630],	||	 recall_10	[0.1750528],	||	 recall_15	[0.2461424]
ndcg_1	[0.4626866],	||	

post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
Training // Epoch 11 //  Total r_cost = 24241.40582  Total a_cost = 1399.88854 total pre adversarial accuracy = 0.00000 total pre adversarial accuracy = 0.00000 total post adversarial accuracy = 0.00000 Training time : 1445 ms negative Sampling time : 809 ms negative samples : 61835
precision_1	[0.4556962],	||	 precision_5	[0.3696203],	||	 precision_10	[0.2974684],	||	 precision_15	[0.2704641]
recall_1   	[0.0348158],	||	 recall_5   	[0.1363150],	||	 recall_10   	[0.2034868],	||	 recall_15   	[0.2689981]
f_measure_1	[0.0646893],	||	 f_measure_5	[0.1991748],	||	 f_measure_10	[0.2416619],	||	 f_measure_15	[0.2697291]
ndcg_1     	[0.4556962],	||	 ndcg_5     	[0.3901517],	||	 ndcg_10     	[0.3584775],	||	 ndcg_

post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post a

...

post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post a

post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post a

post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
Training // Epoch 27 //  Total r_cost = 22805.24286  Total a_cost = 945.08902 total pre adversarial accuracy = 0.00000 total pre adversarial accuracy = 0.00000 total post adversarial accuracy = 0.00000 Training time : 1590 ms negative Sampling time : 866 ms negative samples : 61835
precisi

AUC global is:  0.8678400823033782
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after

post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post adversarial -- adversaire ready after one adv epoch training  0.0
post a

In [ ]:
list_recom_unfairbpr = []
list_recom_constiemfairadvbpr100 = []
list_recom_constiemfairadvbpr200 = []
list_recom_constiemfairadvbpr400 = []
list_recom_constiemfairadvbpr600 = []
list_recom_constiemfairadvbpr800 = []
list_recom_constiemfairadvbpr1000 = []
list_recom_constiemfairadvbpr1200 = []
list_recom_constiemfairadvbpr1500 = []
list_recom_constiemfairadvbpr2000 = []
list_recom_constiemfairadvbpr2500 = []
list_recom_constiemfairadvbpr5000 = []

list_recom_iemfairadvbpr1000 = []

list_recom_fairadvbpr = []

list_recom_iemfairadvbpr = []
list_recom_uemfairbpr = []

list_acc_AdvPre100 = []
list_acc_AdvPost100 = []

list_acc_AdvPre200 = []
list_acc_AdvPost200 = []

list_acc_AdvPre400 = []
list_acc_AdvPost400 = []

list_acc_AdvPre600 = []
list_acc_AdvPost600 = []

list_acc_AdvPre800 = []
list_acc_AdvPost800 = []

list_acc_AdvPre1000 = []
list_acc_AdvPost1000 = []

list_acc_AdvPre1200 = []
list_acc_AdvPost1200 = []

list_acc_AdvPre1500 = []
list_acc_AdvPost1500 = []

list_acc_AdvPre2000 = []
list_acc_AdvPost2000 = []

list_acc_AdvPre2500 = []
list_acc_AdvPost2500 = []

list_acc_AdvPre5000 = []
list_acc_AdvPost5000 = []



for itr in range(1, 40 + 1):
    filename = './unfairBPR_results/epoch'+ str(itr) +'_Rec_' + dataname + '_BPR.npy'
    with open(filename, "rb") as f:
        recom = np.load(f)
        list_recom_unfairbpr.append(recom)

for itr in range(1, 21 + 1):
    filename2 = './fairAdvBPR_results/epoch'+ str(itr) +'_Rec_' + dataname + '_fairAdvBPR.npy'
    with open(filename2, "rb") as f:
        recom = np.load(f)
        list_recom_fairadvbpr.append(recom)
        
for j in adv_const:
    for itr in range(1, train_epoch + train_epoch_a + train_epoch_all + 1):
        filename = './const_occu_UembfairAdvBPRresultsAdvConst'+ str(j) + '/epoch'+ str(itr) +'_pre_' + dataname + '_accAdvpre.npy'
        filename2 = './const_occu_UembfairAdvBPRresultsAdvConst'+ str(j) + '/epoch'+ str(itr) +'_post_' + dataname + '_accAdvpost.npy'
        filename3 = './const_occu_UembfairAdvBPRresultsAdvConst'+ str(j) + '/epoch'+ str(itr) +'_Rec_' + dataname + '_constfairAdvBPR.npy'


        with open(filename, "rb") as f:
            accPre = np.load(f)
            globals()[f"list_acc_AdvPre{j}"].append(accPre)
            #list_acc_AdvPre+str(j)+.append(accPre)

        with open(filename2, "rb") as f:
            accPost = np.load(f)
            globals()[f"list_acc_AdvPost{j}"].append(accPost)
            #list_acc_AdvPost.append(accPost)

        with open(filename3, "rb") as f:
            recom = np.load(f)
            globals()[f"list_recom_constiemfairadvbpr{j}"].append(recom)
            #list_recom_constiemfairadvbpr.append(recom)
        


print("pre ",list_acc_AdvPre1000)
print("\npost ",list_acc_AdvPost1000) 

In [ ]:
# with open('Rec_' + dataname + '_fairAdvBPR.npy', "rb") as f:
#     Recom = np.load(f)
# Recom.shape

In [ ]:
# [precision, recall, f_score, NDCG] = utility.test_model_all(Recom, test_df, train_df)

# utility.test_model_per_user_type(Recom, test_df, train_df, user_type_list, key_type)
# auc = utility.auc_per_user(Recom, test_df, train_df)
# print("AUC global is: ", auc)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from colour import Color

def savepdf_barplot_color_gradient2(ymin = 0.5, ymax = 0.7, whis = 5, start_color='pink',end_color='blue',num_color=5, title='',axis_x = None, xlabel = '', axis_y1 = None, ylabel ='', plot_file = ''):
    
    fig = plt.figure()
    gs = fig.add_gridspec(1, 2, hspace=0, wspace=0)
    (ax1, ax2) = gs.subplots(sharex='col', sharey='row')
    
    red = Color(start_color)
    colors = list(red.range_to(Color(end_color), num_color))
    colors = [color.rgb for color in colors]
    
    X_axis = np.arange(len(axis_x))
    ax1.bar(X_axis, axis_y1, color=colors)
    ax1.hlines(y=axis_y1[0], xmin = 0, xmax = len(axis_x)-1, colors='black', linestyles='--', lw=1)
    
    plt.sca(ax1)
    plt.xticks(X_axis, axis_x, rotation =50)
    #plt.xlabel(xlabel)
    #fig.suptitle(title)
    plt.ylabel(ylabel, fontsize=18)
    plt.rcParams.update({'font.size': 13}) 
    plt.grid()
    
    plt.sca(ax2)
    ax2.boxplot(axis_y1, whis = whis)
    ax1.set_ylim(ymin, ymax)
    
    plt.tight_layout()
    plt.savefig(plot_file)

def savepdf_barplot_color_gradient(ymin = 0.5, ymax = 0.7, start_color='orange',end_color='yellow',num_color=3, title='',axis_x = None, xlabel = '', axis_y1 = None, axis_y2 = None, axis_y3 = None, ylabel ='',barwidth = 0.3, plot_file = '',legendy1 = '',legendy2 = '',legendy3 = ''):
    
    fig, ax1 = plt.subplots()
#     gs = fig.add_gridspec(1, 2, hspace=0, wspace=0)
#     (ax1, ax2) = gs.subplots(sharex='col', sharey='row')
    #fig.suptitle('Sharing x per column, y per row')
    
#     ax1.plot(x + 1, -y, 'tab:green')
#     ax2.plot(x + 2, -y**2, 'tab:red')

#     for ax in fig.get_axes():
#         ax.label_outer()
    
    red = Color(start_color)
    colors = list(red.range_to(Color(end_color), num_color))
    colors = [color.rgb for color in colors]
    
    X_axis = np.arange(len(axis_x))
    #ax1.bar(X_axis, axis_y1, color=colors)
    ax1.bar(2*X_axis - barwidth, axis_y1, width =barwidth, color=colors[num_color-num_color], label = legendy1)
    ax1.bar(2*X_axis + 0, axis_y2, width = barwidth, color=colors[num_color-2],label = legendy2)
    ax1.bar(2*X_axis + barwidth, axis_y3, width = barwidth, color=colors[num_color-1],label = legendy3)
    
    ax1.set_ylim(ymin, ymax)

    #plt.xlabel(xlabel, fontsize=22)
    plt.ylabel(ylabel, fontsize=18)
    #fig.suptitle(title)
    plt.grid()
    plt.legend(fontsize=12) 
    plt.rcParams.update({'font.size': 12})   
    #ax2 = ax1.twinx()
    #ax1.boxplot([[axis_y1[0],axis_y2[0],axis_y3[0]],[axis_y1[1],axis_y2[1],axis_y3[1]],[axis_y1[2],axis_y2[2],axis_y3[2]]], positions=[2*X_axis[0], 2*X_axis[1] , 2*X_axis[2] ])
    ax1.boxplot([[axis_y1[0],axis_y2[0],axis_y3[0]],[axis_y1[1],axis_y2[1],axis_y3[1]],[axis_y1[2],axis_y2[2],axis_y3[2]],[axis_y1[3],axis_y2[3],axis_y3[3]],[axis_y1[4],axis_y2[4],axis_y3[4]]], positions=[2*X_axis[0], 2*X_axis[1] , 2*X_axis[2], 2*X_axis[3], 2*X_axis[4]])
   
    #ax1.set_xticklabels(axis_x)
    plt.xticks(2*X_axis, axis_x)
    
    plt.tight_layout()
    plt.savefig(plot_file)
    
    
    
    
def savepdf_barplot_color_gradient3(ymin = 0.5, ymax = 0.7, title='',axis_x = None, xlabel = '', axis_y = None, ylabel ='',barwidth = 0.3, plot_file = ''):
    
    fig, ax1 = plt.subplots()
    
    X_axis = np.arange(len(axis_x))
    #ax1.bar(X_axis, axis_y1, color=colors)
    #ax1.bar(2*X_axis - barwidth, axis_y1, width =barwidth, color=colors[num_color-num_color], label = legendy1)
    ax1.bar(2*X_axis + 0, axis_y, width = barwidth, color='orange')
    #ax1.bar(2*X_axis + barwidth, axis_y3, width = barwidth, color=colors[num_color-1],label = legendy3)
    
    ax1.set_ylim(ymin, ymax)

    plt.ylabel(ylabel, fontsize=18)
    #fig.suptitle(title)
    plt.grid()
    plt.legend(fontsize=12) 
    plt.rcParams.update({'font.size': 12})   
    plt.xticks(2*X_axis, axis_x)
    
    plt.tight_layout()
    plt.savefig(plot_file)
    
def plotline_save_as_pdf(title='',axis_x = None, xlabel = '', axis_y1 = None, axis_y2 = None , axis_y3 = None, axis_y4 = None,axis_y5 = None, ylabel ='', linewidth=3, plot_file = '',legendy1='',legendy2='',legendy3='',legendy4='',legendy5=''):
    if axis_y1 is not None:
        plt.plot(axis_x, axis_y1, color='orange', linewidth = linewidth, label = legendy1)
    if axis_y2 is not None:
        plt.plot(axis_x, axis_y2, 'g', linewidth = linewidth, label = legendy2)
    if axis_y3 is not None:
        plt.plot(axis_x, axis_y3, 'b', linewidth = linewidth, label = legendy3)
    if axis_y4 is not None:
        plt.plot(axis_x, axis_y4, 'black', linewidth = linewidth, label = legendy4)
    if axis_y5 is not None:
        plt.plot(axis_x, axis_y5, 'red', linewidth = linewidth, label = legendy5)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid()

    plt.savefig(plot_file)

def plotline_save_as_pdf2(title='',axis_x1 = None,axis_x2 = None, xlabel = '', axis_y1 = None, axis_y2 = None , axis_y3 = None, axis_y4 = None,axis_y5 = None, ylabel ='', linewidth=3, plot_file = '',legendy1='',legendy2='',legendy3='',legendy4='',legendy5=''):
    if axis_y1 is not None:
        plt.plot(axis_x1, axis_y1, color='tab:orange', linewidth = linewidth, label = legendy1)
    if axis_y2 is not None:
        plt.plot(axis_x2, axis_y2, color='tab:green', linewidth = linewidth, label = legendy2)
    if axis_y3 is not None:
        plt.plot(axis_x2, axis_y3, color='tab:blue', linewidth = linewidth, label = legendy3)
    if axis_y4 is not None:
        plt.plot(axis_x2, axis_y4, color='tab:purple', linewidth = linewidth, label = legendy4)
    if axis_y5 is not None:
        plt.plot(axis_x2, axis_y5, color='tab:red', linewidth = linewidth, label = legendy5)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid()

    plt.savefig(plot_file)


In [ ]:
auc_plot_unfairbpr=[]
auc_plot_constiemfairbpr=[]
auc_plot_constiemfairbpr1000=[]
auc_plot_constiemfairbpr200=[]
auc_plot_constiemfairbpr600=[]
auc_plot_constiemfairbpr2500=[]
auc_plot_fairadvbpr=[]
auc_plot_iemfairadvbpr=[]
auc_plot_uemfairadvbpr=[]

precision_plot_unfairbpr=[]
precision_plot_constiemfairbpr=[]
precision_plot_constiemfairbpr600=[]
precision_plot_constiemfairbpr2500=[]
precision_plot_constiemfairbpr200=[]
precision_plot_fairadvbpr=[]
precision_plot_iemfairadvbpr=[]
precision_plot_uemfairadvbpr=[]

recall_plot_unfairbpr=[]
recall_plot_constiemfairbpr=[]
recall_plot_constiemfairbpr600=[]
recall_plot_constiemfairbpr2500=[]
recall_plot_constiemfairbpr200=[]
recall_plot_fairadvbpr=[]
recall_plot_iemfairadvbpr=[]
recall_plot_uemfairadvbpr=[]

ndcg_plot_unfairbpr=[]
ndcg_plot_constiemfairbpr=[]
ndcg_plot_constiemfairbpr600=[]
ndcg_plot_constiemfairbpr2500=[]
ndcg_plot_constiemfairbpr200=[]
ndcg_plot_fairadvbpr=[]
ndcg_plot_iemfairadvbpr=[]
ndcg_plot_uemfairadvbpr=[]

precision_perUserType_plot_unfairbpr=[]
precision_perUserType_plot_constiemfairbpr=[]
precision_perUserType_plot_constiemfairbpr600=[]
precision_perUserType_plot_constiemfairbpr2500=[]
precision_perUserType_plot_constiemfairbpr200=[]
precision_perUserType_plot_fairadvbpr=[]
precision_perUserType_plot_iemfairadvbpr=[]
precision_perUserType_plot_uemfairadvbpr=[]

recall_perUserType_plot_unfairbpr=[]
recall_perUserType_plot_constiemfairbpr=[]
recall_perUserType_plot_constiemfairbpr600=[]
recall_perUserType_plot_constiemfairbpr2500=[]
recall_perUserType_plot_constiemfairbpr200=[]
recall_perUserType_plot_fairadvbpr=[]
recall_perUserType_plot_iemfairadvbpr=[]
recall_perUserType_plot_uemfairadvbpr=[]

ndcg_perUserType_plot_unfairbpr=[]
ndcg_perUserType_plot_constiemfairbpr=[]
ndcg_perUserType_plot_constiemfairbpr600=[]
ndcg_perUserType_plot_constiemfairbpr2500=[]
ndcg_perUserType_plot_constiemfairbpr200=[]
ndcg_perUserType_plot_fairadvbpr=[]
ndcg_perUserType_plot_iemfairadvbpr=[]
ndcg_perUserType_plot_uemfairadvbpr=[]

auc_perUserType_plot_unfairbpr=[]
auc_perUserType_plot_constiemfairbpr=[]
auc_perUserType_plot_constiemfairbpr600=[]
auc_perUserType_plot_constiemfairbpr2500=[]
auc_perUserType_plot_constiemfairbpr200=[]
auc_perUserType_plot_fairadvbpr=[]
auc_perUserType_plot_iemfairadvbpr=[]
auc_perUserType_plot_uemfairadvbpr=[]

In [ ]:


for rec in list_recom_unfairbpr:
    [precision, recall, f_score, ndcg] = utility.test_model_all(rec, test_df, train_df)

    [precision_per_user_type, recall_per_user_type, ndcg_per_user_type, auc_per_user_type] = utility.test_model_per_user_type(rec, test_df, train_df, user_type_list, key_type)
    auc = utility.auc_per_user(rec, test_df, train_df)
    auc_plot_unfairbpr.append(auc)
    precision_plot_unfairbpr.append(precision)
    recall_plot_unfairbpr.append(recall)
    ndcg_plot_unfairbpr.append(ndcg)
    
    precision_perUserType_plot_unfairbpr.append(precision_per_user_type)
    recall_perUserType_plot_unfairbpr.append(recall_per_user_type)
    ndcg_perUserType_plot_unfairbpr.append(ndcg_per_user_type)
    auc_perUserType_plot_unfairbpr.append(auc_per_user_type)
    
for rec in list_recom_constiemfairadvbpr1000:
    [precision, recall, f_score, ndcg] = utility.test_model_all(rec, test_df, train_df)

    [precision_per_user_type, recall_per_user_type, ndcg_per_user_type, auc_per_user_type] = utility.test_model_per_user_type(rec, test_df, train_df, user_type_list, key_type)
    auc = utility.auc_per_user(rec, test_df, train_df)
    auc_plot_constiemfairbpr.append(auc)
    precision_plot_constiemfairbpr.append(precision)
    recall_plot_constiemfairbpr.append(recall)
    ndcg_plot_constiemfairbpr.append(ndcg)
    
    precision_perUserType_plot_constiemfairbpr.append(precision_per_user_type)
    recall_perUserType_plot_constiemfairbpr.append(recall_per_user_type)
    ndcg_perUserType_plot_constiemfairbpr.append(ndcg_per_user_type)
    auc_perUserType_plot_constiemfairbpr.append(auc_per_user_type)

for rec in list_recom_fairadvbpr:
    [precision, recall, f_score, ndcg] = utility.test_model_all(rec, test_df, train_df)

    [precision_per_user_type, recall_per_user_type, ndcg_per_user_type, auc_per_user_type] = utility.test_model_per_user_type(rec, test_df, train_df, user_type_list, key_type)
    auc = utility.auc_per_user(rec, test_df, train_df)
    auc_plot_fairadvbpr.append(auc)
    precision_plot_fairadvbpr.append(precision)
    recall_plot_fairadvbpr.append(recall)
    ndcg_plot_fairadvbpr.append(ndcg)
    
    precision_perUserType_plot_fairadvbpr.append(precision_per_user_type)
    recall_perUserType_plot_fairadvbpr.append(recall_per_user_type)
    ndcg_perUserType_plot_fairadvbpr.append(ndcg_per_user_type)
    auc_perUserType_plot_fairadvbpr.append(auc_per_user_type)

    


In [ ]:
for rec in list_recom_constiemfairadvbpr1000:
    [precision, recall, f_score, ndcg] = utility.test_model_all(rec, test_df, train_df)

    [precision_per_user_type, recall_per_user_type, ndcg_per_user_type, auc_per_user_type] = utility.test_model_per_user_type(rec, test_df, train_df, user_type_list, key_type)
    auc = utility.auc_per_user(rec, test_df, train_df)
    auc_plot_constiemfairbpr.append(auc)
    precision_plot_constiemfairbpr.append(precision)
    recall_plot_constiemfairbpr.append(recall)
    ndcg_plot_constiemfairbpr.append(ndcg)
    
    precision_perUserType_plot_constiemfairbpr.append(precision_per_user_type)
    recall_perUserType_plot_constiemfairbpr.append(recall_per_user_type)
    ndcg_perUserType_plot_constiemfairbpr.append(ndcg_per_user_type)
    auc_perUserType_plot_constiemfairbpr.append(auc_per_user_type)

In [ ]:
#savepdf_barplot_color_gradient(ymin = 0.7, ymax = 0.9, start_color='orange',end_color='olive',num_color=3, title='AUC per User Type on Testing (Movielens)',axis_x = ['UnfairBPR','Us1000','fairAdvBPR','Us600','Us2500'], xlabel = '', axis_y1 = [auc_plot_unfairbpr[-1],auc_plot_constiemfairbpr[-1],auc_plot_fairadvbpr[-20] ,auc_plot_constiemfairbpr600[-1], auc_plot_constiemfairbpr2500[-1]], axis_y2 = [auc_perUserType_plot_unfairbpr[-1]['M'],auc_perUserType_plot_constiemfairbpr[-1]['M'],auc_perUserType_plot_fairadvbpr[-1]['M'],auc_perUserType_plot_constiemfairbpr600[-1]['M'],auc_perUserType_plot_constiemfairbpr2500[-1]['M']], axis_y3 = [auc_perUserType_plot_unfairbpr[-1]['F'],auc_perUserType_plot_constiemfairbpr[-1]['F'],auc_perUserType_plot_fairadvbpr[-1]['F'],auc_perUserType_plot_constiemfairbpr600[-1]['F'],auc_perUserType_plot_constiemfairbpr2500[-1]['F']], ylabel ='AUC',barwidth = 0.3, plot_file = './unfairBPR_plots/Uemb_auc_per_type.pdf',legendy1='Overall',legendy2='Male',legendy3='Female') 
savepdf_barplot_color_gradient(ymin = 0.7, ymax = 0.91, start_color='blue',end_color='gray',num_color=3, title='AUC per User Type on Testing (Movielens)',axis_x = ['UnfairBPR','Us200','Us600','Us1000','Us2500'], xlabel = '', axis_y1 = [auc_plot_unfairbpr[-1],auc_plot_constiemfairbpr[-1] ,auc_plot_constiemfairbpr[-1],auc_plot_constiemfairbpr[-1], auc_plot_constiemfairbpr[-1]], axis_y2 = [auc_perUserType_plot_unfairbpr[-1]['engineer'],auc_perUserType_plot_constiemfairbpr[-1]['engineer'],auc_perUserType_plot_constiemfairbpr[-1]['engineer'],auc_perUserType_plot_constiemfairbpr[-1]['engineer'],auc_perUserType_plot_constiemfairbpr[-1]['engineer']], axis_y3 = [auc_perUserType_plot_unfairbpr[-1]['scientist'],auc_perUserType_plot_constiemfairbpr[-1]['scientist'],auc_perUserType_plot_constiemfairbpr[-1]['scientist'],auc_perUserType_plot_constiemfairbpr[-1]['scientist'],auc_perUserType_plot_constiemfairbpr[-1]['scientist']], ylabel ='AUC',barwidth = 0.3, plot_file = './unfairBPR_plots/Uemb_auc_per_type.pdf',legendy1='Overall',legendy2='engineer',legendy3='scientist') 